# Download Imdb data from
 http://mng.bz/0tIo

let’s collect the individual training reviews into a list of strings, one string per
review. We’ll also collect the review labels (positive/negative) into a labels list. 

In [1]:
# Lets take a look at the downloaded dada

import os

print(os.listdir('aclImdb'))
print(os.listdir('aclImdb/train'))

['.DS_Store', 'test', 'train']
['.DS_Store', 'neg', 'pos', 'urls_neg.txt', 'urls_pos.txt']


In [2]:
# IMDB Dataset contain negative and positive reviews in text format 

In [3]:
f = open("aclImdb/train/pos/268_8.txt", "r")
print(f.read())

I loved Dedee Pfeiffer (is that spelled right?) in Cybil. Haven't seen her for awhile and forgot how much I missed her. I thought she did a great job in this. The supporting cast was pretty good too. In some angles, the daughter even looked like a young Nicole Kidman. The abductor was pretty creepy and the story generally had some good twists. The young boyfriend was a hottie. I thought the husband definitely had something to do with it for sure.<br /><br />Just got the Lifetime Movie Network for Christmas and am loving these movies. Kept my interest and I'll watch it again when they rerun it. Can anyone else recommend any similar movies to this? You can post on the board or send me a private email if you want. Thanks in advance. Aboutagirly.


In [4]:
f = open("aclImdb/train/neg/10002_1.txt", "r")
print(f.read())

Sorry everyone,,, I know this is supposed to be an "art" film,, but wow, they should have handed out guns at the screening so people could blow their brains out and not watch. Although the scene design and photographic direction was excellent, this story is too painful to watch. The absence of a sound track was brutal. The loooonnnnng shots were too long. How long can you watch two people just sitting there and talking? Especially when the dialogue is two people complaining. I really had a hard time just getting through this film. The performances were excellent, but how much of that dark, sombre, uninspired, stuff can you take? The only thing i liked was Maureen Stapleton and her red dress and dancing scene. Otherwise this was a ripoff of Bergman. And i'm no fan f his either. I think anyone who says they enjoyed 1 1/2 hours of this is,, well, lying.


# Processing the labels of the raw IMDB data

In [5]:
# With this entire code we just created labels for the neg and pos text

import os
imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train') #train_dir = aclImdb\train
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type) #dir_name = aclImdb\train\neg
                                                   #dir_name = aclImdb\train\pos
        
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':                  # if file is text file
            f = open(os.path.join(dir_name, fname),encoding="utf8") # open it
            texts.append(f.read())               # add in that empty texts list we made before
            f.close()
        if label_type == 'neg':                 # ager neg folder ka ho to 0 label
            labels.append(0)                    # aur labels list mai daldo jo upper bnaya tha
        else:
            labels.append(1)                   # pos hoto 1 label deke labels folder mai daldo

# Tokenizing the text of the raw IMDB data
we’ll add the following twist:
restricting the training data to the first 200 samples. So you’ll learn to classify movie
reviews after looking at just 200 examples.

In [6]:
maxlen = 100               # Cuts off reviews after 100 words
training_samples = 200     # Trains on 200 samples
validation_samples = 10000 # Validates on 10,000 samples
max_words = 10000          # Considers only the top 10,000 words in the dataset

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_words)    # max_words = 10000 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [8]:
print('Found %s unique tokens.' % len(tokenizer.word_index))
print()
#print(tokenizer.word_index)

Found 88582 unique tokens.



In [9]:
# pad_sequences is used to ensure that all sequences in a list have the same length.

from tensorflow.keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=maxlen)                    # maxlen = 100   

In [10]:
import numpy as np

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [11]:
# randomizing data

indices = np.arange(data.shape[0])   # same length ke range bnai 
np.random.shuffle(indices)           # uske position shuffle ke
data = data[indices]                 # same shuffle position data ke bhe kerde
labels = labels[indices]             # aur label ke bhe

In [12]:
# spliting data

x_train = data[:training_samples]  # training_samples = 200 
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples] # validation_samples = 10000
y_val = labels[training_samples: training_samples + validation_samples]

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(200, 100)
(200,)
(10000, 100)
(10000,)


# DOWNLOAD THE GLOVE WORD EMBEDDINGS
Its a pretrained Word Embedding : 
https://nlp.stanford.edu/projects/glove

It’s an 822 MB zip file called glove.6B.zip,
containing 100-dimensional embedding vectors for 400,000 words (or nonword
tokens).

PREPROCESSING THE EMBEDDINGS
Let’s parse the unzipped file (a .txt file) to build an index that maps words (as strings)
to their vector representation (as number vectors).

# Parsing the GloVe word-embeddings file

In [13]:
# lets check the downloaded glove file

import os
print("Folder contain :",os.listdir("glove.6B"))
print()

# each file colntain word with its vextor lets check

f = open('glove.6B/glove.6B.100d.txt',encoding="utf8")
for i in f:
    values = i.split()
f.close()
print("Printing first value :\n\n",values)

Folder contain : ['glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.6B.50d.txt']

Printing first value :

 ['sandberger', '0.28365', '-0.6263', '-0.44351', '0.2177', '-0.087421', '-0.17062', '0.29266', '-0.024899', '0.26414', '-0.17023', '0.25817', '0.097484', '-0.33103', '-0.43859', '0.0095799', '0.095624', '-0.17777', '0.38886', '0.27151', '0.14742', '-0.43973', '-0.26588', '-0.024271', '0.27186', '-0.36761', '-0.24827', '-0.20815', '0.22128', '-0.044409', '0.021373', '0.24594', '0.26143', '0.29303', '0.13281', '0.082232', '-0.12869', '0.1622', '-0.22567', '-0.060348', '0.28703', '0.11381', '0.34839', '0.3419', '0.36996', '-0.13592', '0.0062694', '0.080317', '0.0036251', '0.43093', '0.01882', '0.31008', '0.16722', '0.074112', '-0.37745', '0.47363', '0.41284', '0.24471', '0.075965', '-0.51725', '-0.49481', '0.526', '-0.074645', '0.41434', '-0.1956', '-0.16544', '-0.045649', '-0.40153', '-0.13136', '-0.4672', '0.18825', '0.2612', '0.16854', '0.22615', '0.62992', '-0

In [14]:
# lets put the same format in a dictionary

glove_dir = 'glove.6B'                                                   # ?
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'),encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))
print()
#print(embeddings_index)

Found 400000 word vectors.



Next, you’ll build an embedding matrix that you can load into an Embedding layer. It
must be a matrix of shape (max_words, embedding_dim), where each entry i contains
the embedding_dim-dimensional vector for the word of index i in the reference word
index (built during tokenization). Note that index 0 isn’t supposed to stand for any
word or token—it’s a placeholder.

# Preparing the GloVe word-embeddings matrix

In [15]:
# to be continued